In [183]:
import numpy as np
import pandas as pd
import json

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score, classification_report

import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression

## Image Label (Neural Network)

In [671]:
class Net(nn.Module):

    def __init__(self, INPUT_SIZE):
        super(Net, self).__init__()
        # Input is output of resnet18 image classification model
        self.fc1 = nn.Linear(INPUT_SIZE, 750) 
        self.fc11 = nn.Linear(750, 250)
        self.fc2 = nn.Linear(250, 100)
        self.fc3 = nn.Linear(100, 1)
        self.act_out = nn.Sigmoid()

        self.dropout = nn.Dropout(p=0.25)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc11(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.act_out(self.fc3(x))
        return x


def load_dataset_df(label_path):
    label = pd.read_csv(label_path)
    label = label.drop(columns=['Unnamed: 0'])
    dataset = label

    # load embeddings
    with open('./dataset_work/embeddings.json') as f:
        embeddings = json.load(f)
    all_embeddings = pd.DataFrame.from_dict(embeddings, orient='index', columns=['embedding'])
    all_embeddings.index.name = 'id'
    dataset = pd.merge(dataset, all_embeddings, on='id', how='left')

    # load personalities
    with open('./dataset_work/personalities.json') as f:
        personalities = json.load(f)
    all_personalities = pd.DataFrame.from_dict(personalities, orient='index', columns=['playfulness', 'chase-proneness', 'curiosity', 'sociability', 'aggressiveness', 'shyness'])
    all_personalities.index.name = 'id'
    dataset = pd.merge(dataset, all_personalities, on='id', how='left')

    return dataset

def vectorize_dataset(df, with_personalities, with_embeddings):
    if with_personalities and with_embeddings:
        cols = df[['embedding','playfulness', 'chase-proneness', 'curiosity', 'sociability', 'aggressiveness', 'shyness']]
        x = cols.values
        embeddings = x[:,0]
        expanded = np.array(embeddings.tolist())
        x = np.concatenate((expanded, x[:,1:]), axis=1)
        x = np.array(x, dtype='float64')
        y = df['label'].values
    elif with_embeddings:
        cols = df[['embedding','playfulness', 'chase-proneness', 'curiosity', 'sociability', 'aggressiveness', 'shyness']]
        x = cols.values
        embeddings = x[:,0]
        expanded = np.array(embeddings.tolist())
        x = expanded
        y = df['label'].values
    else:
        cols = df[['embedding','playfulness', 'chase-proneness', 'curiosity', 'sociability', 'aggressiveness', 'shyness']]
        x = cols.values
        x = x[:,1:]
        x = np.array(x, dtype="float64")
        y = df['label'].values
    return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

def load(label_path, with_personalities, with_embeddings):
    df = load_dataset_df(label_path)
    return vectorize_dataset(df, with_personalities, with_embeddings)

def nn_pred(label_path):
    # load the weights, trained on only images
    PATH = "trained_models/BEST_MODEL_ALICE/alice_medium_images_True_personalities_False_optm_adam_loss_MSE_EPOCHS_100_BATCH_1000_REG_0.095_LR_0.0001_f1_0.788327619047619"
    train_x_t, train_y_t = load(label_path, with_personalities=False, with_embeddings=True)
    model = Net(train_x_t.shape[1])
    model.load_state_dict(torch.load(PATH))
    model.eval()
    return model(train_x_t)

In [672]:
nn_preds = nn_pred('dataset_work/labels/image_only/alice_valid_personalityFalse_imageTrue_labels.csv')
valid_i = load_dataset_df('./dataset_work/labels/image_only/alice_valid_personalityFalse_imageTrue_labels.csv')
#valid_i = load_dataset_df('./dataset_work/labels/image_and_personality/alice_valid_personalityFalse_imageTrue_labels.csv')

In [673]:
print(classification_report(nn_preds.round().detach().numpy(), valid_i['label']))

              precision    recall  f1-score   support

         0.0       0.80      0.75      0.78        81
         1.0       0.59      0.66      0.62        44

    accuracy                           0.72       125
   macro avg       0.70      0.71      0.70       125
weighted avg       0.73      0.72      0.72       125



In [565]:
len(nn_preds)

125

## Personality Label (Decision Trees)

In [566]:
def load_dataset_p(label_path):
    label = pd.read_csv(label_path)
    label = label.drop(columns=['Unnamed: 0'])
    p_file = open('./dataset_work/personalities.json')
    personalities = json.load(p_file)
    p_file.close()
    all_personalities = pd.DataFrame.from_dict(personalities, orient='index', columns=['playfulness', 'chase-proneness', 'curiosity', 'sociability', 'aggressiveness', 'shyness'])
    all_personalities['id'] = all_personalities.index
    dataset = pd.merge(label, all_personalities, on='id', how='left')
    return dataset

In [567]:
# laod the personality datasets
train_p = load_dataset_p('./dataset_work/labels/personality_only/alice_train_personalityTrue_imageFalse_labels.csv')
valid_p = load_dataset_p('./dataset_work/labels/personality_only/alice_valid_personalityTrue_imageFalse_labels.csv')
test_p = load_dataset_p('./dataset_work/labels/personality_only/alice_test_personalityTrue_imageFalse_labels.csv')

In [568]:
def split_dataset(d_df):
    _feats = d_df[['playfulness', 'chase-proneness', 'curiosity', 'sociability', 'aggressiveness', 'shyness']]
    x = _feats.values
    return x, d_df['label'].values

In [569]:
X_train_p, y_train_p = split_dataset(train_p)
X_valid_p, y_valid_p = split_dataset(valid_p)
X_test_p, y_test_p = split_dataset(test_p)

In [570]:
# add regularization with 'lambda'
personality_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42, n_estimators=50, scale_pos_weight=0.8, reg_lambda=2.2)
personality_model.fit(X_train_p, y_train_p)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=50, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=42, ...)

In [571]:
valid_preds = personality_model.predict(X_valid_p)
train_preds = personality_model.predict(X_train_p)
test_preds = personality_model.predict(X_test_p)
#p_probs = [v[1] for v in valid_preds] # probability of a output label '1'

In [572]:
print(classification_report(train_preds, y_train_p))

              precision    recall  f1-score   support

           0       0.98      0.95      0.96        94
           1       0.98      0.99      0.99       281

    accuracy                           0.98       375
   macro avg       0.98      0.97      0.97       375
weighted avg       0.98      0.98      0.98       375



In [573]:
print(classification_report(valid_preds, y_valid_p))

              precision    recall  f1-score   support

           0       0.84      0.70      0.76        37
           1       0.88      0.94      0.91        88

    accuracy                           0.87       125
   macro avg       0.86      0.82      0.84       125
weighted avg       0.87      0.87      0.87       125



In [574]:
print(classification_report(test_preds, y_test_p))

              precision    recall  f1-score   support

           0       0.80      0.88      0.84        32
           1       0.96      0.92      0.94        92

    accuracy                           0.91       124
   macro avg       0.88      0.90      0.89       124
weighted avg       0.92      0.91      0.91       124



## Putting them together on images + personalities

In [575]:
def gen_final_data(nn_path, p_model, d_b, X_b, out):
    # create a 3-vector [breed, NN output, tree output] for the training set  
    nn_preds = nn_pred(nn_path)
    preds = p_model.predict_proba(X_b) # using the model trained on only personality data
    p_preds = [v[1] for v in preds]
    dog_ids = d_b['id'].values
    breed_ids = [i[1:i.find('_')] for i in dog_ids]
    
    dog_ids = d_b['id'].values
    breed_ids = [i[1:i.find('_')] for i in dog_ids]
    breed_to_a = {}
    ctr = 0
    for breed in breed_ids:
        if breed not in breed_to_a:
            breed_to_a[breed] = ctr
            ctr += 1

    processed_breed_ids = [breed_to_a[b] for b in breed_ids]
    
    #out['breed'] = processed_breed_ids
    out['p_out'] = p_preds
    out['img_out'] = nn_preds.detach().numpy()
    

    for breed in breed_to_a.keys():
        nc = []
        for i in breed_ids:
            if i == breed:
                #print('y')
                nc.append(1)
            else:
                nc.append(0)
        out[breed] = nc
 

In [576]:
train_b = load_dataset_df('./dataset_work/labels/image_and_personality/alice_train_personalityTrue_imageTrue_labels.csv')
valid_b = load_dataset_df('./dataset_work/labels/image_and_personality/alice_valid_personalityTrue_imageTrue_labels.csv')
test_b = load_dataset_df('./dataset_work/labels/image_and_personality/alice_test_personalityTrue_imageTrue_labels.csv')

X_train_b, y_train_b = split_dataset(train_b)
X_valid_b, y_valid_b = split_dataset(valid_b)
X_test_b, y_test_b = split_dataset(test_b)

In [577]:
BOTH_TRAIN_PATH = './dataset_work/labels/image_and_personality/alice_train_personalityTrue_imageTrue_labels.csv'
BOTH_VALID_PATH = './dataset_work/labels/image_and_personality/alice_valid_personalityTrue_imageTrue_labels.csv'
BOTH_TEST_PATH = './dataset_work/labels/image_and_personality/alice_test_personalityTrue_imageTrue_labels.csv'
train_master = pd.DataFrame()
valid_master = pd.DataFrame()
test_master = pd.DataFrame()
gen_final_data(BOTH_TRAIN_PATH, personality_model, train_b, X_train_b, train_master)
gen_final_data(BOTH_VALID_PATH, personality_model, valid_b, X_valid_b, valid_master)
gen_final_data(BOTH_TEST_PATH, personality_model, test_b, X_test_b, test_master)

train_master_X = train_master.values
valid_master_X = valid_master.values
test_master_X = test_master.values

In [578]:
test_master

,p_out,img_out,02097209,02099267,02091467,02087046,02096585,02106382,02107908,02091635,...,02088364,02100236,02094114,02115641,02099712,02100877,02106030,02113624,02110185,02093647
0,0.988195,0.467600,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.057259,0.321074,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.752446,0.500520,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.972088,0.349371,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.986096,0.490736,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,0.987721,0.463072,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
120,0.183292,0.308596,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
121,0.542351,0.455900,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
122,0.998880,0.256798,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [662]:
master_model = xgb.XGBClassifier(objective="binary:logistic",  booster='gbtree', random_state=42, n_estimators=30, scale_pos_weight=0.95, reg_lambda=2)
#master_model = LogisticRegression()
master_model.fit(train_master_X, y_train_b)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=30, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=42, ...)

In [663]:
sum(y_train_b) / len(y_train_b)

0.41333333333333333

In [664]:
final_train_preds = master_model.predict(train_master_X)

In [665]:
print(classification_report(y_train_b, final_train_preds))

              precision    recall  f1-score   support

           0       0.93      0.95      0.94       220
           1       0.92      0.90      0.91       155

    accuracy                           0.93       375
   macro avg       0.93      0.92      0.93       375
weighted avg       0.93      0.93      0.93       375



In [666]:
final_valid_preds = master_model.predict(valid_master_X)

In [667]:
print(classification_report(y_valid_b, final_valid_preds))

              precision    recall  f1-score   support

           0       0.72      0.73      0.73        79
           1       0.53      0.52      0.53        46

    accuracy                           0.66       125
   macro avg       0.63      0.63      0.63       125
weighted avg       0.65      0.66      0.66       125



In [668]:
final_test_preds = master_model.predict(test_master_X)

In [669]:
sum(final_test_preds) / len(final_test_preds)

0.41935483870967744

In [670]:
print(classification_report(y_test_b, final_test_preds))

              precision    recall  f1-score   support

           0       0.78      0.75      0.76        75
           1       0.63      0.67      0.65        49

    accuracy                           0.72       124
   macro avg       0.71      0.71      0.71       124
weighted avg       0.72      0.72      0.72       124



In [634]:
# Vrushank to-do: 
# refactor data, use k-fold